In [1]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from numba import njit, prange

In [87]:
N: int = 512
ITER: int = 4

@njit
def IX(x: int, y: int) -> int:
    x = min(max(x, 0), N - 1)
    y = min(max(y, 0), N - 1)
    return x + y * N

In [88]:
@dataclass
class Fluid:
    dt: float
    diffusion: float
    viscosity: float

    s: np.ndarray
    density: np.ndarray

    Vx: np.ndarray
    Vy: np.ndarray

    Vx0: np.ndarray
    Vy0: np.ndarray

    def __init__(self, diffusion: float, viscosity: float, dt: float):
        self.dt = dt
        self.diffusion = diffusion
        self.viscosity = viscosity

        self.s = np.zeros(N * N, dtype=float)
        self.density = np.zeros(N * N, dtype=float)

        self.Vx = np.zeros(N * N, dtype=float)
        self.Vy = np.zeros(N * N, dtype=float)

        self.Vx0 = np.zeros(N * N, dtype=float)
        self.Vy0 = np.zeros(N * N, dtype=float)

    def add_density(self, x: int, y: int, amount: float):
        self.density[IX(x, y)] += amount

    def add_velocity(self, x: int, y: int, amount_x: float, amount_y: float):
        index: int = IX(x, y)
        self.Vx[index] += amount_x
        self.Vy[index] += amount_y
        
    def step(self) -> None:
        visc = self.viscosity
        diff = self.diffusion
        dt = self.dt
        Vx = self.Vx
        Vy = self.Vy
        Vx0 = self.Vx0
        Vy0 = self.Vy0
        s = self.s
        density = self.density
    
        diffuse(1, Vx0, Vx, visc, dt)
        diffuse(2, Vy0, Vy, visc, dt)
    
        project(Vx0, Vy0, Vx, Vy)

        advect(1, Vx, Vx0, Vx0, Vy0, dt)
        advect(2, Vy, Vy0, Vx0, Vy0, dt)

        project(Vx, Vy, Vx0, Vy0)

        diffuse(0, s, density, diff, dt)

        advect(0, density, s, Vx, Vy, dt)

In [89]:
@njit(fastmath=True)
def set_bnd(b: int, x: np.ndarray) -> None:
    mult2 = (-1 if b == 2 else 1)
    mult1 = (-1 if b == 1 else 1)    
    
    # x[1: N - 1] = x[1 + (1) * N: N -1 + (1) * N] * mult2
    # x[1 + (N - 1) * N: N - 1 + (N - 1) * N] = x[1 + (N - 2) * N: N - 1 + (N - 2) * N] * mult2
    
    for i in range(1, N - 1):
        x[IX(i, 0)] = x[IX(i, 1)] * mult2
        x[IX(i, N - 1)] = x[IX(i, N - 2)] * mult2

    
    for j in range(1, N - 1):
        x[IX(0, j)] = x[IX(1, j)] * mult1
        x[IX(N - 1, j)] = x[IX(N - 2, j)] * mult1

    x[IX(0, 0)] = 0.5 * (x[IX(1, 0)] + x[IX(0, 1)])
    x[IX(0, N - 1)] = 0.5 * (x[IX(1, N - 1)] + x[IX(0, N - 2)])
    x[IX(N - 1, 0)] = 0.5 * (x[IX(N - 2, 0)] + x[IX(N - 1, 1)])
    x[IX(N - 1, N - 1)] = 0.5 * (x[IX(N - 2, N - 1)] + x[IX(N - 1, N - 2)])

In [90]:
@njit(fastmath=True, parallel=True)
def lin_solve(b: int, x: np.ndarray, x0: np.ndarray, a: float, c: float) -> None:
    c_recip: float = 1.0 / c
    for k in range(0, ITER):
        for j in prange(1, N - 1):
            for i in range(1, N - 1):
                x[IX(i, j)] = (x0[IX(i, j)]
                               + a * (x[IX(i + 1, j)]
                                      + x[IX(i - 1, j)]
                                      + x[IX(i, j + 1)]
                                      + x[IX(i, j - 1)]
                                      )) * c_recip

        set_bnd(b, x)


def diffuse(b: int, x: np.ndarray, x0: np.ndarray, diffusion: float, dt: float) -> None:
    a: float = dt * diffusion * (N - 2) * (N - 2)
    lin_solve(b, x, x0, a, 1 + 4 * a)


In [91]:
@njit(fastmath=True, parallel=True)
def project(veloc_x: np.ndarray, veloc_y: np.ndarray, p: np.ndarray, div: np.ndarray) -> None:
    for j in prange(1, N - 1):
        for i in range(1, N - 1):
            div[IX(i, j)] = -0.5 * (
                    veloc_x[IX(i + 1, j)]
                    - veloc_x[IX(i - 1, j)]
                    + veloc_y[IX(i, j + 1)]
                    - veloc_y[IX(i, j - 1)]
            ) / N
            p[IX(i, j)] = 0

    set_bnd(0, div)
    set_bnd(0, p)
    lin_solve(0, p, div, 1, 4)

    for j in prange(1, N - 1):
        for i in range(1, N - 1):
            veloc_x[IX(i, j)] -= 0.5 * (p[IX(i + 1, j)] - p[IX(i - 1, j)]) * N
            veloc_y[IX(i, j)] -= 0.5 * (p[IX(i, j + 1)] - p[IX(i, j - 1)]) * N

    set_bnd(1, veloc_x)
    set_bnd(2, veloc_y)


In [92]:
@njit(fastmath=True)
def advect(b: int, d: np.ndarray, d0: np.ndarray, veloc_x: np.ndarray, veloc_y: np.ndarray, dt: float) -> None:
    dtx: float = dt * (N - 2)
    dty: float = dt * (N - 2)

    Nfloat: float = N
    
    for j in range(1, N - 1):
        jfloat: float = float(j)
        for i in range(1, N - 1):
            ifloat: float = float(i)

            tmp1: float = dtx * veloc_x[IX(i, j)]
            tmp2: float = dty * veloc_y[IX(i, j)]

            x: float = ifloat - tmp1
            y: float = jfloat - tmp2

            x: float = min(max(x, 0.5), Nfloat + 0.5)

            i0: float = np.floor(x)
            i1: float = i0 + 1.0

            y: float = min(max(y, 0.5), Nfloat + 0.5)

            j0: float = np.floor(y)
            j1: float = j0 + 1.0

            s1: float = x - i0
            s0: float = 1.0 - s1
            t1: float = y - j0
            t0: float = 1.0 - t1

            i0i: int = int(i0)
            i1i: int = int(i1)
            j0i: int = int(j0)
            j1i: int = int(j1)

            d[IX(i, j)] = (
                    s0 * (t0 * d0[IX(i0i, j0i)] + t1 * d0[IX(i0i, j1i)]) +
                    s1 * (t0 * d0[IX(i1i, j0i)] + t1 * d0[IX(i1i, j1i)])
            )

    set_bnd(b, d)

In [93]:
SCALE = 2

In [94]:
old_pos_x = 0
old_pos_y = 0

def handle_mouse_movement(position):
    global old_pos_x, old_pos_y
    pos_x, pos_y = int(np.round(position[1] / SCALE)), int(np.round(position[0] / SCALE))
    
    density = 100
    
    velocity_x = pos_x - old_pos_x
    velocity_y = pos_y - old_pos_y
    
    fluid.add_density(pos_x, pos_y, density)
    fluid.add_velocity(pos_x, pos_y, velocity_x, velocity_y)
    if pos_x - 1 > 0: 
        fluid.add_density(pos_x - 1, pos_y, density)
        fluid.add_velocity(pos_x - 1, pos_y, velocity_x, velocity_y)
    if pos_y - 1 > 0: 
        fluid.add_density(pos_x, pos_y - 1, density)
        fluid.add_velocity(pos_x, pos_y - 1, velocity_x, velocity_y)
    if pos_x + 1 < N: 
        fluid.add_density(pos_x + 1, pos_y, density)
        fluid.add_velocity(pos_x + 1, pos_y, velocity_x, velocity_y)
    if pos_y + 1 < N: 
        fluid.add_density(pos_x, pos_y + 1, density)
        fluid.add_velocity(pos_x, pos_y + 1, velocity_x, velocity_y)

    # if pos_x - 1 > 0 and pos_y - 1 > 0: 
    #     fluid.add_density(pos_x - 1, pos_y - 1, density)
    #     fluid.add_velocity(pos_x - 1, pos_y - 1, velocity_x, velocity_y)
    # if pos_x - 1 > 0 and pos_y + 1 < N: 
    #     fluid.add_density(pos_x - 1, pos_y + 1, density)
    #     fluid.add_velocity(pos_x - 1, pos_y + 1, velocity_x, velocity_y)
    # if pos_x + 1 < N and pos_y + 1 < N: 
    #     fluid.add_density(pos_x + 1, pos_y + 1, density)
    #     fluid.add_velocity(pos_x + 1, pos_y + 1, velocity_x, velocity_y)
    # if pos_x + 1 < N and pos_y - 1 > 0: 
    #     fluid.add_density(pos_x + 1, pos_y - 1, density)
    #     fluid.add_velocity(pos_x + 1, pos_y - 1, velocity_x, velocity_y)

    old_pos_x, old_pos_y = pos_x, pos_y
    # print(position)
    # print(fluid.density)
    # print(position)
    # pass

In [95]:
@njit(fastmath=True)
def resize_density(density: np.ndarray, scale: int) -> np.ndarray:
    if scale == 1: 
        return density.reshape((N, N))
    resized_density = np.empty((N * scale * N * scale), dtype=float)
    for j in range(0, N):
        for i in range(0, N):
            den: float = density[IX(i, j)]
            for sj in range(scale):
                for si in range(scale):
                    index = i * scale + si + (j * scale + sj) * N * scale
                    resized_density[index] = den
    return resized_density.reshape((N * scale, N * scale))

In [100]:
import cv2
import pygame
from IPython.display import Image, display
import time
from datetime import timedelta

fluid: Fluid = Fluid(0.00000, 0.00000, 0.15)
pygame.init()
screen = pygame.display.set_mode((N * SCALE, N * SCALE))
running = True

try:
  while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEMOTION:
            # Update simulation based on mouse movement
        
            handle_mouse_movement(event.pos)

    # Update and render your simulation
    start_time = time.monotonic()
    fluid.step()
    end_time = time.monotonic()
    # print("--- %s seconds ---" % (time.time() - start_time))
    # print(end_time - start_time)
    print(f"Duration: {int((end_time - start_time) * 1000)}ms")
    # numpy_array = get_updated_simulation()
    density_scaled = cv2.resize(fluid.density.reshape(N, N), dsize=(N * SCALE, N * SCALE), interpolation=cv2.INTER_LINEAR)
    
    # density_scaled = resize_density(fluid.density, SCALE)
    pygame.surfarray.blit_array(screen, density_scaled)  # Assumes numpy_array is a 2D array representing pixel colors
    pygame.display.flip()
except:
  print("An exception occurred")

# 
# pygame.display.flip()
# pygame.image.save(screen, 'frame.png')
# display(Image(filename='frame.png'))

pygame.quit()

Duration: 30ms
Duration: 16ms
Duration: 15ms
Duration: 30ms
Duration: 15ms
Duration: 15ms
Duration: 16ms
Duration: 15ms
Duration: 16ms
Duration: 31ms
Duration: 15ms
Duration: 15ms
Duration: 15ms
Duration: 32ms
Duration: 15ms
Duration: 16ms
Duration: 15ms
Duration: 15ms
Duration: 15ms
Duration: 32ms
Duration: 16ms
Duration: 15ms
Duration: 31ms
Duration: 15ms
Duration: 15ms
Duration: 15ms
Duration: 15ms
Duration: 32ms
Duration: 30ms
Duration: 16ms
Duration: 15ms
Duration: 15ms
Duration: 15ms
Duration: 16ms
Duration: 16ms
Duration: 30ms
Duration: 15ms
Duration: 15ms
Duration: 16ms
Duration: 15ms
Duration: 15ms
Duration: 30ms
Duration: 15ms
Duration: 15ms
Duration: 16ms
Duration: 15ms
Duration: 15ms
Duration: 15ms
Duration: 32ms
Duration: 16ms
Duration: 15ms
Duration: 15ms
Duration: 15ms
Duration: 16ms
Duration: 30ms
Duration: 16ms
Duration: 15ms
Duration: 30ms
Duration: 15ms
Duration: 15ms
Duration: 16ms
Duration: 15ms
Duration: 15ms
Duration: 30ms
Duration: 32ms
Duration: 15ms
Duration: 

In [69]:
import cv2
import pygame
from IPython.display import Image, display
import time
from datetime import timedelta
import math

fluid: Fluid = Fluid(0.00001, 0.000001, 0.15)
pygame.init()
screen = pygame.display.set_mode((N * SCALE, N * SCALE))
running = True

def draw_arrow(screen, color, start, end):
    pygame.draw.line(screen, color, start, end, 2)
    rotation = math.degrees(math.atan2(start[1]-end[1], end[0]-start[0])) + 90
    # pygame.draw.polygon(screen, color, ((end[0] + 5 * math.sin(math.radians(rotation)), end[1] + 5 * math.cos(math.radians(rotation))), 
    #                                     (end[0] + 5 * math.sin(math.radians(rotation - 120)), end[1] + 5 * math.cos(math.radians(rotation - 120))), 
    #                                     (end[0] + 5 * math.sin(math.radians(rotation + 120)), end[1] + 5 * math.cos(math.radians(rotation + 120)))))

try:
  while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEMOTION:
            # Update simulation based on mouse movement
        
            handle_mouse_movement(event.pos)

    # Update and render your simulation
    start_time = time.monotonic()
    fluid.step()
    end_time = time.monotonic()
    # print("--- %s seconds ---" % (time.time() - start_time))
    # print(end_time - start_time)
    print(f"Duration: {int((end_time - start_time) * 1000)}ms")
    # numpy_array = get_updated_simulation()
    # density_scaled = cv2.resize(fluid.density.reshape(N, N), dsize=(N * SCALE, N * SCALE), interpolation=cv2.INTER_LINEAR)
    screen.fill((255, 255, 255))
    velocity_field = np.concatenate(
        (fluid.Vx.reshape((N, N, 1)), fluid.Vy.reshape((N, N, 1))),
        axis=-1
    )
    
    vector_length = 5
    max_vector_length = 5
    
    for i, row in enumerate(velocity_field):
        for j, vec in enumerate(row):
            # Scale and position each vector
            start_pos = (i * (N * SCALE) // velocity_field.shape[0], j * (N * SCALE) // velocity_field.shape[1])
            end_pos = (start_pos[0] + min(vec[0] * SCALE *vector_length, max_vector_length), start_pos[1] + min(vec[1] * SCALE *vector_length, max_vector_length))
            
            draw_arrow(screen, (0, 0, 0), start_pos, end_pos)

    
    # density_scaled = resize_density(fluid.density, SCALE)
    # pygame.surfarray.blit_array(screen, density_scaled)  # Assumes numpy_array is a 2D array representing pixel colors
    pygame.display.flip()
except:
  print("An exception occurred")

# 
# pygame.display.flip()
# pygame.image.save(screen, 'frame.png')
# display(Image(filename='frame.png'))

pygame.quit()

Duration: 0ms
Duration: 15ms
Duration: 15ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 15ms
Duration: 0ms
Duration: 15ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 15ms
Duration: 0ms
Duration: 15ms
Duration: 0ms
Duration: 15ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 16ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 15ms
Duration: 0ms
Duration: 15ms
Duration: 15ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 15ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 15ms
Duration: 0ms
Duration: 0ms
Duration: 0ms
Duration: 15ms
Durati

In [19]:
fluid.Vx.reshape((N, N, 1)).shape

(512, 512, 1)